## 🌟 Looking at SAR Beamforming from a Different Angle

Sometimes, approaching the same problem from a **different perspective** can reveal exciting insights. So far, we’ve tackled **SAR image formation as a beamforming process**, where we combine signals from the **virtual antenna elements** with appropriate **phase shifts**. 📡✨  

But what if we look at it through the lens of **correlation processing**? Let’s find out! 🔍🤔  

---

### 🔍 Matched Filtering & Correlation Processing  
When we performed **range compression**, we used **matched filtering via cross-correlation**. 🧩

The idea was simple: The output at a certain time delay was obtained by **correlating the received echo signal with the transmitted chirp**. 🎵➕🎵  

Now, what if we apply this same concept to **SAR beamforming**? 🔦  

---

### 📏📐 Beamforming as Correlation Processing  
For a given pixel in the image, we calculate a **reference signal** — essentially a phasor with a different value for each virtual antenna element (pulse) with the **opposite phase** of what we expect the echo from that pixel location to have.  

By **multiplying the received signal with this reference phasor**, we ensure the echoes from the desired pixel are **aligned and summed coherently**. 💪💥  

But there's a catch! 🎣

---

### 🍌 The Banana-Shaped Curve of SAR Beamforming  
Unlike range compression, where each virtual element only records a **1D time series of voltages**, things get more complicated with beamforming:  

- In **range compression**, we applied the matched filter **separately for each pulse**. That means we could perform operations on 1D signals.  
- But now, because of our **high range resolution and changing distances** between the radar and the target, the echoes appear at **different time delays for each virtual antenna element**. 📡⏳  
- These time delays trace out that delicious **banana-shaped curve** in the data. 🍌✨  

---

### 🔑 **How to Perform Correlation Processing**  
When we approach **beamforming as correlation processing**, we’re essentially calculating the **match between the received data and the expected signal from a specific pixel location**. 🧩 Here's how we do it:  

Imagine a **hypothetical point target** located precisely at the pixel position we want to focus on. 🎯

- Ideally, the echo from this target would produce an **extremely narrow, impulse-like pulse in range**.  
- As the radar moves along its flight path, this pulse traces out the tasty **banana-shaped curve** in the 2D data matrix (range vs. pulse number). 🍌✨  

### 🔨 **The Trick: Correlation Processing**  
For a **single pixel location** 📍, here's how we can calculate the correlation:

1. **Generate the 2D Reference Signal:**  
   - Simulate a narrow pulse signal including the **correct time delay and phase** based on the distance between the radar and the pixel location at each pulse. ↘️
   - As a result, we get a **simulated data matrix** that has the **same dimensions** as our measured echo data. 🔢

2. **Multiply With the Data:**  
   - Multiply the received signal by the **conjugate** (to subtract the phase) of this simulated data, effectively applying a **2D mask** that picks out the relevant samples.  ✖️
   - This step also **corrects the phase** of the echo data from the pixel location to allow coherent summation. ↗️➕↗️ 

3. **Sum the Aligned Samples:**  
   - Sum all the phase-aligned samples to obtain the **pixel phasor**—the focused SAR image sample for that specific pixel location. ↗️  

To obtain the full **single-look-complex SAR image**, we just **repeat this process** for each pixel location in the image. 🖼️

### 💡 **What’s Happening Here:**  
As you may have noticed, calculating the **correlation between this 2D pixel reference signal and the received echo data** performs exactly the same operations as the **backprojection beamformer**. 📐✨  

This approach allows us to **isolate the desired pixel’s contribution from the rest of the data**, effectively performing **beamforming by correlation processing**! 🖼️📡  

---

### 📡🔦 Two Ways of Looking at the Same Problem 🧩⚙️
The correlation approach is just another way to describe the same principle. The **physics** is identical. You can view SAR image formation from either perspective:  

- **Beamforming:** Focusing the beam of the large virtual antenna array on each pixel. 📡🔦  
- **Correlation Processing:** Calculating the correlation between the echo data and the expected signal from each pixel. 🧩⚙️

---

### 🎮 **Interactive Experiment: Simulating the Pixel Response**  
Now, let’s put this concept to the test! 🔧🚀  

- You can **input the (x, y) coordinates on the ground** representing the **target pixel** you want to focus on. 🎯  
- The code will **simulate the echo response** from that specific pixel as the radar moves along its flight path.  
- This simulation produces a **pulsed signal return** that traces the **banana-shaped curve of time delays (ranges)** in the 2D data matrix. 🍌🗺️  

---

### ⏳ **Watch the Echo Phasor Evolve:**  
In this experiment, you can **visualize how the phasor of the pixel reference signal changes over time** as the radar moves from one measurement position to the next:  
- 📈 The **range changes most rapidly** at the beginning, as the radar approaches the target.  
- 🐢 As the radar moves past the target, the rate of change **gradually slows down**.  
- 🔄 Right in the **middle of the aperture**, the direction of the range change **reverses**.  

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 100  # set animation size limit (MB)


# Define target pixel position
target_pos=(0.0, 0.0, 0.0)
target_x, target_y, target_z = target_pos

range_resolution = 1.0 # m
num_pulses = 500 # number of pulses in simulation
wavelength = 10.0 # m 

aperture_length = 1000 # length of synthetic aperture m
y_radar = 1000.0 # m
z_radar = 1000.0 # m
x_start = -aperture_length/2 # Aperture start (m)
x_end = aperture_length/2 # Aperture end (m)

# Calculate min and max ranges
range_margin = 50 # Include some extra range buffer in the data
range_max = np.sqrt((x_start - target_x)**2 + (y_radar - target_y)**2 + (z_radar - target_z)**2) + range_margin
range_min = np.max([np.sqrt(target_x**2 + (y_radar - target_y)**2 + (z_radar - target_z)**2) - range_margin, 0.0])
num_range_samples = int(round(np.abs(range_max - range_min)/range_resolution))

# 1) Generate radar positions
x_positions = np.linspace(x_start, x_end, num_pulses)
radar_positions = np.column_stack([
    x_positions,
    np.full(num_pulses, y_radar),
    np.full(num_pulses, z_radar)
])

# 2) Prepare range axis & create mask array
range_axis = np.linspace(range_min, range_max, num_range_samples)
range_res = range_axis[1] - range_axis[0]
mask = np.zeros((num_pulses, num_range_samples))


# We'll store the actual distance for each pulse
range_values = np.zeros(num_pulses, dtype=np.float32)

# 3) Compute slant range and fill mask with 2-bin interpolation
for i in range(num_pulses):
    dx = radar_positions[i, 0] - target_x
    dy = radar_positions[i, 1] - target_y
    dz = radar_positions[i, 2] - target_z
    r = np.sqrt(dx**2 + dy**2 + dz**2)
    range_values[i] = r

    # distribute amplitude between two neighboring range bins
    range_bin = (r - range_min) / range_res  # floating-point bin index
    bin_floor = int(np.floor(range_bin))
    bin_ceil  = bin_floor + 1

    if bin_floor >= 0 and bin_floor < num_range_samples:
        alpha = range_bin - bin_floor
        mask[i, bin_floor] += (1.0 - alpha)
    if bin_ceil >= 0 and bin_ceil < num_range_samples:
        alpha = range_bin - bin_floor
        mask[i, bin_ceil]  += alpha

# Plot the raw data (banana curve)
plt.figure(figsize=(7, 5))
plt.imshow(
    mask,
    aspect='auto',
    cmap='jet',
    extent=[range_axis[0], range_axis[-1], num_pulses, 0]
)
plt.title("Reference Data Matrix")
plt.xlim([range_axis[0], range_axis[-1]])
plt.xlabel("Range (m)")
plt.ylabel("Pulse Number")
plt.colorbar(label="Amplitude")
plt.show()

# 5) Compute reference echo signal: s(i) = exp(-j * k * r(i))
k = 2.0 * np.pi / wavelength
reference_signal = np.exp(-1j * k * range_values)

# Plot Real & Imag in one time-series
real_part = np.real(reference_signal)
imag_part = np.imag(reference_signal)

plt.figure(figsize=(8, 4))
plt.plot(real_part, label="I Component")
plt.plot(imag_part, label="Q Component")
plt.title("Reference Signal (I & Q)")
plt.xlabel("Pulse Number")
plt.ylabel("Amplitude")
plt.legend()
plt.tight_layout()
plt.show()

real_part = np.real(reference_signal)
imag_part = np.imag(reference_signal)

# Create a figure with 2 subplots: left -> phasor, right -> range vs. pulse
fig, (ax_phasor, ax_range) = plt.subplots(1, 2, figsize=(10, 5))

# Phasor arrow from origin
arrow = FancyArrowPatch(
    posA=(0,0),
    posB=(0,0),
    arrowstyle='->',
    mutation_scale=10,
    color='red',
    linewidth=2
)
ax_phasor.add_patch(arrow)

max_amp = np.abs(reference_signal).max()
margin = 0.1 * max_amp
ax_phasor.set_xlim([-max_amp - margin, max_amp + margin])
ax_phasor.set_ylim([-max_amp - margin, max_amp + margin])
ax_phasor.set_aspect('equal', 'box')
ax_phasor.set_xlabel("I Component")
ax_phasor.set_ylabel("Q Component")
ax_phasor.set_title("Reference Phasor vs. Pulse")

# Range vs. pulse subplot
line_range, = ax_range.plot([], [], 'b-', lw=2)
point_range, = ax_range.plot([], [], 'ro', ms=6)
ax_range.set_xlim([0, num_pulses])
ax_range.set_ylim([range_values.min() - 10, range_values.max() + 10])
ax_range.set_xlabel("Pulse Index")
ax_range.set_ylabel("Range (m)")
ax_range.set_title("Range vs. Pulse")

def init():
    arrow.set_positions((0,0), (0,0))
    line_range.set_data([], [])
    point_range.set_data([], [])
    return arrow, line_range, point_range

def update(frame):
    # Update phasor arrow
    x_end = real_part[frame]
    y_end = imag_part[frame]
    arrow.set_positions((0,0), (x_end, y_end))

    # Update range line
    x_arr = np.arange(frame+1)
    y_arr = range_values[:frame+1]
    line_range.set_data(x_arr, y_arr)

    # Mark the current sample with a red dot
    point_range.set_data([frame], [range_values[frame]])

    return arrow, line_range, point_range

anim = animation.FuncAnimation(
    fig,
    update,
    frames=num_pulses,
    init_func=init,
    interval=50,
    blit=False
)

HTML(anim.to_jshtml())

### 🔍 **What’s Happening with the Reference Signal Phase?** 📡✨  

When you examine how the **phase of the reference signal changes**, you’ll notice something interesting. 📈👀  

It looks a lot like a **chirp!** 🐦🎶 But there's a twist:  
- In our radar pulse waveform, a chirp is a signal that **varies in frequency over time**. ⏳🔄  
- Here, the “chirp” is a signal that **varies across space**—specifically, **across the synthetic aperture**. 🌌📡
- The phase of the signal changes from one pulse to the next because the **distance between the radar and the target** has changed.

There’s one other difference: 🌀 
  - A true chirp signal has a **quadratic phase change over time** (linearly changing frequency).  
  - The phase change we see here is actually **hyperbolic**, which is close to quadratic but not exactly the same.  

What we’re seeing is like a **spatial chirp**, where the phase variation happens across the **synthetic aperture** instead of over time. 🛰️   

This is why the **phase of the reference signal** looks familiar—it’s behaving similarly to a chirp, but in a different context than before! 🐦